## 1. Define functions for... 

* collecting real time data from binance futures markets for any timeframe and numbers of candlesticks (1000 max)
* calculating support and resistance 

In [25]:
def collect_data(timeframe='4h', limit=500):
    # This function downloads candlestick data from
    # binance futures market

    # define the market
    exchange_f = ccxt.binance({
        'enableRateLimit': True,
        'options': {
            'defaultType': 'future',  # or 'margin'
        }})

    all_coins_f = list(exchange_f.load_markets().keys())

    coins = [x for x in all_coins_f if "BUSD" not in x]
    coins = [x for x in coins if "_" not in x]

    all_candles_f = []
    for symbol in coins:
        df = pd.DataFrame(exchange_f.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit))
        df['symbol'] = symbol

        df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']

        df['Datetime'] = df['Datetime'].apply(
            lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))
        all_candles_f.append(df)

    all_candles_f = pd.concat(all_candles_f)

    return all_candles_f

def detect_levels(df):
    s =  np.mean(df['High'] - df['Low'])

    def isSupport(df,i):
      support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]
      return support
    def isResistance(df,i):
      resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2]
      return resistance

    def isFarFromLevel(l):
       return np.sum([abs(l-x) < s  for x in levels]) == 0

    levels = []
    for i in range(2,df.shape[0]-2):
      if isSupport(df,i):
        l = df['Low'][i]
        if isFarFromLevel(l):
          levels.append((i,l))
      elif isResistance(df,i):
        l = df['High'][i]
        if isFarFromLevel(l):
          levels.append((i,l))
    return levels

## 2. Apply functions in practice

In [71]:
import numpy as np
import pandas as pd
import ccxt
import time
import dateutil
from datetime import datetime
from functools import reduce
from scipy.signal import argrelextrema
#from functions import *

import plotly.graph_objects as go

# Pull datafrom exchange
data = collect_data(timeframe = '1d', limit = 200)

In [122]:
#for i in data['Symbol'].unique().tolist(): this code will loop through all cryptos which takes too much time and generates a large file

for i in ["BTC/USDT", "ETH/USDT", "BNB/USDT"]:
    
    print(i)
    df = data[data['Symbol'] == i].tail(100)
    df = df.reset_index()
    dt = df.Datetime
    
    levels = detect_levels(df)
    times = []
    prices = []
    for level in levels:
        time = df['Datetime'][level[0]]
        price = level[1]
        times.append(time)
        prices.append(price)

    test = pd.DataFrame(data = {'levels':prices, 'dates':times})

    print(test)
    candlestick = go.Candlestick(x=df['Datetime'], open=df['Open'], high=df['High'], low=df['Low'],
                                 close=df['Close'])

    fig = go.Figure(data=[candlestick])

    for row in test.iterrows():
        fig.add_shape(type="line", x0= row[1]['dates'], 
                                y0 = row[1]['levels'],
                                x1 = dt.max(),
                                y1 = row[1]['levels'], fillcolor = 'yellow')
    
    fig.layout.xaxis.type = 'category'

    fig.update_layout(
        width=800,
        height=600)

    fig.layout.xaxis.rangeslider.visible = False
    fig.show()

BTC/USDT
     levels                dates
0  57550.00  2021-10-28 00:00:00
1  64349.55  2021-11-02 00:00:00
2  69198.70  2021-11-10 00:00:00
3  51929.00  2021-12-07 00:00:00
4  45650.00  2021-12-31 00:00:00
5  41666.00  2022-01-14 00:00:00


ETH/USDT
   levels                dates
0  4145.0  2021-11-01 00:00:00
1  4676.0  2021-11-03 00:00:00
2  3580.0  2021-12-30 00:00:00


BNB/USDT
   levels                dates
0  429.90  2021-10-27 00:00:00
1  662.80  2021-11-10 00:00:00
2  516.36  2021-11-18 00:00:00
3  616.68  2021-12-09 00:00:00
4  554.46  2021-12-24 00:00:00


## 3. Alternative way to support and resistance levels

* Here I use argrelextrema from scipy package to detect low and high values in a local area as support and resistance levels

In [120]:
def detect_levels_new(df, order=10):
    # this function is to detect latest 10 price peaks (lowest and highest)

    dt = df.Datetime

    df = df.set_index('Datetime')

    price = df.Close

    high = df.High

    low = df.Low

    max_idx = list(argrelextrema(high.values, np.greater, order=order)[0])
    min_idx = list(argrelextrema(low.values, np.less, order=order)[0])

    peak_1 = high.values[max_idx]
    peak_2 = low.values[min_idx]

    peaks_p = list(peak_1) + list(peak_2)

    peaks_idx = list(max_idx) + list(min_idx)

    peaks_idx_dt = np.array(dt.values[peaks_idx])

    peaks_p = np.array(list(peak_1) + list(peak_2))

    final_df = pd.DataFrame({"levels": peaks_p, "datetime": peaks_idx_dt})

    return final_df

In [121]:
for i in ["BTC/USDT", "ETH/USDT", "BNB/USDT"]:
    
    print(i)
    df = data[data['Symbol'] == i].tail(100)
    #df = df.reset_index()
    dt = df.Datetime
    levels = detect_levels_new(df, order=10)

    print(test)
    candlestick = go.Candlestick(x=df['Datetime'], open=df['Open'], high=df['High'], low=df['Low'],
                                 close=df['Close'])

    fig = go.Figure(data=[candlestick])

    for row in levels.iterrows():
        fig.add_shape(type="line", x0= row[1]['datetime'], 
                                y0 = row[1]['levels'],
                                x1 = dt.max(),
                                y1 = row[1]['levels'], fillcolor = 'yellow')
    
    fig.layout.xaxis.type = 'category'

    fig.update_layout(
        width=800,
        height=600)

    fig.layout.xaxis.rangeslider.visible = False
    fig.show()

BTC/USDT
     levels             datetime
0  67150.00  2021-10-20 00:00:00
1  69198.70  2021-11-10 00:00:00
2  52134.00  2021-12-27 00:00:00
3  57550.00  2021-10-28 00:00:00
4  40888.89  2021-12-04 00:00:00
5  45373.11  2021-12-17 00:00:00
6  39610.00  2022-01-10 00:00:00
7  32853.83  2022-01-24 00:00:00


ETH/USDT
     levels             datetime
0  67150.00  2021-10-20 00:00:00
1  69198.70  2021-11-10 00:00:00
2  52134.00  2021-12-27 00:00:00
3  57550.00  2021-10-28 00:00:00
4  40888.89  2021-12-04 00:00:00
5  45373.11  2021-12-17 00:00:00
6  39610.00  2022-01-10 00:00:00
7  32853.83  2022-01-24 00:00:00


BNB/USDT
     levels             datetime
0  67150.00  2021-10-20 00:00:00
1  69198.70  2021-11-10 00:00:00
2  52134.00  2021-12-27 00:00:00
3  57550.00  2021-10-28 00:00:00
4  40888.89  2021-12-04 00:00:00
5  45373.11  2021-12-17 00:00:00
6  39610.00  2022-01-10 00:00:00
7  32853.83  2022-01-24 00:00:00
